In [1]:
import math
import torch
from utils import *
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

import joblib

In [2]:
df = fetch_data()
df_combined = combine_features(df, context=5)
X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(df_combined)
X_train.head()

,date_id,time_id,symbol_id,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,...,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
1157744,112,831,33,NaN,NaN,NaN,NaN,NaN,0.649997,1.250937,...,0.083852,-0.186456,-0.247947,0.107769,-0.238652,-0.309708,-0.161242,-0.344682,-0.322328,-0.291973
131284,15,804,10,NaN,NaN,NaN,NaN,NaN,0.237513,0.622777,...,-0.283100,0.008067,-0.055721,-0.109639,0.106814,0.112933,-0.309250,-0.418137,-0.406175,-0.372990
881903,91,729,34,NaN,NaN,NaN,NaN,NaN,-0.892826,-0.437469,...,0.503609,0.482061,-0.309142,-0.100867,-1.433333,-1.712605,-0.882016,-1.518287,-1.419028,-1.389193
977227,100,2,7,NaN,NaN,NaN,NaN,NaN,-0.959374,-0.367786,...,-0.865782,-0.981885,-0.401543,-0.604826,NaN,NaN,-0.350785,-0.316638,-0.324064,-0.323139
1137021,111,410,3,NaN,NaN,NaN,NaN,NaN,0.493863,-0.052120,...,-0.314514,-0.762610,0.838707,-0.185329,0.258680,0.204036,-0.698816,-0.590934,0.271598,0.350628


In [14]:
X_train.shape

(1749789, 83)

In [15]:
X_train_small = X_train[:100000]
y_train_small = y_train[:100000]

### __Random Forest Regressor__

In [ ]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('model', RandomForestRegressor(random_state=42))
], verbose=1)

pipeline.fit(X_train_small, y_train_small["responder_0"]);

[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.2s
[Pipeline] ............. (step 2 of 2) Processing model, total=25.4min


In [22]:
pipeline2 = joblib.load("models/pipeline.pkl")

In [23]:
Y_pred = pipeline2.predict(X_test)

mse = mean_squared_error(y_test["responder_0"], Y_pred)
print("Mean Squared Error:", mse)

column_range = df['responder_0'].max() - df['responder_0'].min()
print(f"RANGE: \t{column_range}")
print(f"MAX: \t{df['responder_0'].max()}")
print(f"MIN: \t{df['responder_0'].min()}")

Mean Squared Error: 0.767311558678644
RANGE: 	10.0
MAX: 	5.0
MIN: 	-5.0


### __MLP (Multi Layer Perceptron)__
- `Context`: `5` __time steps__

- `Input`: `79` __features__ $\times$ `5` __time steps__ = `395` __features__
- `Output`: __Responder 6__
- `Model`: `3` __hidden layers__ with `256`, `128`, `64` __neurons__
- `Activation`: __ReLU__
- `Regularization`: __L2__
- `Optimizer`: __Adam__
- `Loss`: __Weighted Zero-Mean R-squared Score ($R^2$)__

$$
  R^2 = 1 - \frac{\sum w_i (y_i - \hat{y}_i)^2}{\sum w_i y_i^2}
$$



In [4]:
df.head()

date_id  time_id  symbol_id    weight  feature_00  feature_01  feature_02  \
0        0        0          1  3.889038         NaN         NaN         NaN   
1        0        0          7  1.370613         NaN         NaN         NaN   
2        0        0          9  2.285698         NaN         NaN         NaN   
3        0        0         10  0.690606         NaN         NaN         NaN   
4        0        0         14  0.440570         NaN         NaN         NaN   

   feature_03  feature_04  feature_05  ...  feature_78  responder_0  \
0         NaN         NaN    0.851033  ...   -0.281498     0.738489   
1         NaN         NaN    0.676961  ...   -0.302441     2.965889   
2         NaN         NaN    1.056285  ...   -0.096792    -0.864488   
3         NaN         NaN    1.139366  ...   -0.296244     0.408499   
4         NaN         NaN    0.955200  ...    3.418133    -0.373387   

   responder_1  responder_2  responder_3  responder_4  responder_5  \
0    -0.069556     1.380875     2.005353     0.186018     1.218368   
1     1.190077    -0.523998     3.849921     2.626981     5.000000   
2    -0.280303    -0.326697     0.375781     1.271291     0.099793   
3     0.223992     2.294888     1.097444     1.225872     1.225376   
4    -0.502764    -0.348021    -3.928148    -1.591366    -5.000000   

   responder_6  responder_7  responder_8  
0     0.775981     0.346999     0.095504  
1     0.703665     0.216683     0.778639  
2     2.109352     0.670881     0.772828  
3     1.114137     0.775199    -1.379516  
4    -3.572820    -1.089123    -5.000000  

[5 rows x 92 columns]

In [8]:
df.describe()

date_id       time_id     symbol_id        weight  feature_00  \
count  1.944210e+06  1.944210e+06  1.944210e+06  1.944210e+06         0.0   
mean   9.384629e+01  4.240000e+02  1.376638e+01  1.973281e+00         NaN   
std    4.813196e+01  2.450851e+02  1.108778e+01  9.679003e-01         NaN   
min    0.000000e+00  0.000000e+00  0.000000e+00  4.405696e-01         NaN   
25%    5.400000e+01  2.120000e+02  7.000000e+00  1.323803e+00         NaN   
50%    9.900000e+01  4.240000e+02  1.200000e+01  1.763827e+00         NaN   
75%    1.360000e+02  6.360000e+02  1.700000e+01  2.393846e+00         NaN   
max    1.690000e+02  8.480000e+02  3.800000e+01  6.011999e+00         NaN   

       feature_01  feature_02  feature_03  feature_04    feature_05  ...  \
count         0.0         0.0         0.0         0.0  1.944210e+06  ...   
mean          NaN         NaN         NaN         NaN -4.463175e-02  ...   
std           NaN         NaN         NaN         NaN  9.471079e-01  ...   
min           NaN         NaN         NaN         NaN -1.176608e+01  ...   
25%           NaN         NaN         NaN         NaN -4.756072e-01  ...   
50%           NaN         NaN         NaN         NaN -5.818180e-02  ...   
75%           NaN         NaN         NaN         NaN  3.493771e-01  ...   
max           NaN         NaN         NaN         NaN  1.532000e+01  ...   

         feature_78   responder_0   responder_1   responder_2   responder_3  \
count  1.944210e+06  1.944210e+06  1.944210e+06  1.944210e+06  1.944210e+06   
mean  -9.805073e-02  8.424639e-03  1.076465e-02  2.412764e-03  1.139089e-02   
std    6.392438e-01  9.543021e-01  1.139575e+00  8.429762e-01  1.274226e+00   
min   -3.393299e+00 -5.000000e+00 -5.000000e+00 -5.000000e+00 -5.000000e+00   
25%   -3.195696e-01 -2.322211e-01 -2.667868e-01 -1.210319e-01 -4.444537e-01   
50%   -2.468792e-01 -3.943805e-03 -2.333469e-02 -1.248489e-03 -1.031224e-02   
75%   -1.256560e-01  2.309678e-01  2.538152e-01  1.191232e-01  4.292141e-01   
max    4.370195e+01  5.000000e+00  5.000000e+00  5.000000e+00  5.000000e+00   

        responder_4   responder_5   responder_6   responder_7   responder_8  
count  1.944210e+06  1.944210e+06  1.944210e+06  1.944210e+06  1.944210e+06  
mean   2.185480e-02  3.326982e-03  1.487634e-03 -4.817980e-04  1.078154e-03  
std    1.264991e+00  1.222891e+00  8.696651e-01  8.909110e-01  8.726581e-01  
min   -5.000000e+00 -5.000000e+00 -5.000000e+00 -5.000000e+00 -5.000000e+00  
25%   -5.146699e-01 -2.569897e-01 -3.558709e-01 -3.914038e-01 -2.892584e-01  
50%   -1.836913e-02 -4.690550e-03 -9.597129e-03 -2.376243e-02 -3.107830e-04  
75%    5.065366e-01  2.438874e-01  3.360999e-01  3.463979e-01  2.840819e-01  
max    5.000000e+00  5.000000e+00  5.000000e+00  5.000000e+00  5.000000e+00  

[8 rows x 92 columns]